In [ ]:
####################### Bin.INI : An Ensemble Approach for Dynamic Data Streams #############################
################################# Authors: Muhammad Usman, Huanhuan Chen ####################################
############################## For any queries: muhammadusman@mail.ustc.edu.cn ##############################
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
from skmultiflow.trees import HoeffdingTreeClassifier
from time import perf_counter
from sklearn import clone
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import sys
import scipy.stats as ss
from scipy.stats import ks_2samp
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
import statistics
from decimal import Decimal
import scipy.cluster.hierarchy as sch
from sklearn import preprocessing
from numpy import sqrt
import matplotlib.ticker as mticker
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
import random
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
import math 
from numpy.linalg import norm
import statistics
import random
import os
import warnings
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial  import distance_matrix
from scipy.spatial.distance import pdist,squareform
from scipy.cluster.hierarchy import linkage
import matplotlib.pyplot as plt
import numpy as np
from efficient_apriori import apriori
#from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from math import log2
import uuid
from sklearn.metrics import cohen_kappa_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from pandas.api.types import CategoricalDtype
from pandas._libs.interval import Interval
from timeit import default_timer as timer


pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.8f}'.format)
#np.set_printoptions(suppress=True)
#np.set_printoptions(threshold=1000000000)
warnings.filterwarnings('ignore')


def __get_coefficients(y_true, y_pred_a, y_pred_b):
    a, b, c, d = 0, 0, 0, 0
    for i in range(y_true.shape[0]):
        if y_pred_a[i] == y_true[i] and y_pred_b[i] == y_true[i]:
            a = a + 1
        elif y_pred_a[i] != y_true[i] and y_pred_b[i] == y_true[i]:
            b = b + 1
        elif y_pred_a[i] == y_true[i] and y_pred_b[i] != y_true[i]:
            c = c + 1
        else:
            d = d + 1
 
    return a, b, c, d

def q_statistics(y_true, y_pred_a, y_pred_b):
    a, b, c, d = __get_coefficients(y_true, y_pred_a, y_pred_b)
    q = float(a * d - b * c) / (a * d + b * c)
    return q

def double_fault_measure(y_true, y_pred_a, y_pred_b):
    a, b, c, d = __get_coefficients(y_true, y_pred_a, y_pred_b)
    df = float(d) / (a + b + c + d)
    return df

def disagreement_measure(y_true, y_pred_a, y_pred_b):
    a, b, c, d = __get_coefficients(y_true, y_pred_a, y_pred_b)
    disagreement = float(b + c) / (a + b + c + d)
    return disagreement


def custom_transform(original_value):
    a = 1.0
    transformed_value = a * (1 - original_value)
    return transformed_value

def cal_diversities(y_true, y_pred_a, y_pred_b):
    
    a, b, c, d = __get_coefficients(y_true, y_pred_a, y_pred_b)
    
    q = float(a * d - b * c) / (a * d + b * c)
    
    df = float(d) / (a + b + c + d)
    
    da = float(b + c) / (a + b + c + d)
    
    #print("original-")
    #print(q)
    #print(df) 
    #print(da)   
    q = custom_transform(q)
    df = 1 - df
    #print("some error")
    #print("transformed-")
    ##print(q)
    #print(df) 
    #print(da)   
    # Find the minimum and maximum values
    min_value = min(q, df, da)
    max_value = max(q, df, da)

    # Normalize each value
    q = min_max_normalize(q, min_value, max_value)
    df = min_max_normalize(df, min_value, max_value)
    da = min_max_normalize(da, min_value, max_value)
    
    #print("normalized-")
    #print(q)
    #print(df) 
    #print(da)   
    return q,df,da


def min_max_normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += str(ele)+ "-" 
    
    # return string  
    return str1 
# normalize a vector to have unit norm
def normalize(weights):
    # calculate l1 vector norm
    result = norm(weights, 1)
    # check for a vector of all zeros
    if result == 0.0:
        return weights
    # return normalized vector (unit norm)
    return weights / result
def pltcolor(lst):
    cols=[]
    for l in lst:
        if l==0:
            cols.append('blue')
        elif l==1:
            cols.append('red')
    return cols


def detect_overlap(data_min,data_maj,n_features):
    fishers = []
    found = 0
    max_f1 = 0
    max_feature_index = 0
    Xy_up = data_min
    Xy_down = data_maj
    frames = [Xy_up,Xy_down]
    Xy_d = pd.concat(frames)
    f_data_up_means = Xy_up.mean()
    f_data_down_means= Xy_down.mean()
    f_data_up_vars = Xy_up.var()
    f_data_down_vars =Xy_down.var()
    for i in range(n_features):
        mean_up = f_data_up_means[i]
        mean_down = f_data_down_means[i]
        var_up = f_data_up_vars[i]
        var_down = f_data_down_vars[i]
        fisher = ((mean_up-mean_down)**2) / (var_up + var_down)
        fishers.append([i,round(fisher,5),round(mean_up,5),round(mean_down,5)])
        if(fisher>max_f1) :
            max_f1 = fisher
            max_feature_index = i

    max_index= max_feature_index + 1
    #print("Actual Max:",max_f1)
    max_f1 = 1/(1+max_f1)   # push between (0,1]
    #print("Transformed Max:",max_f1)       
    fishers=sorted(fishers,key=lambda x: x[1], reverse=False)
    return round(max_f1,5), fishers

def update_minority_buffer(X,y,n_features):
    global minority_buffer_x
    global minority_buffer_y 
    global w
    global df_buffer_x
    global min_life_max
    
    decrease_life()   
    remove_zero_life_rows()
    for n in range (0, len(X)):
        df_buffer_x = df_buffer_x.append(X.iloc[n])
        df_buffer_x.iloc[-1,df_buffer_x.columns.get_loc("life")] = min_life_max
        df_buffer_x.iloc[-1,df_buffer_x.columns.get_loc("recall_weight")] = 1
        df_buffer_x.iloc[-1,df_buffer_x.columns.get_loc("total_weight")] = 1
    recalculate_total_weight()

def decrease_life():
    global df_buffer_x
    global min_life_decay_factor    
    df_buffer_x['life'] -= min_life_decay_factor
   # print(df_buffer_x)

def update_recall_weight(weight):
    global df_buffer_x
    global min_life_max
    df_buffer_x.loc[df_buffer_x.life == min_life_max, 'recall_weight'] = weight   
    #print(df_buffer_x)
   # sys.exit()
    
def recalculate_total_weight():
    global df_buffer_x
    global min_life_max
    df_buffer_x['total_weight'] = 0.5* df_buffer_x['life'] + 0.5* df_buffer_x['recall_weight']   
    #print(df_buffer_x)
    
#performs random oversampling  
def resample_from_buffer(X,y,samples_to_add):
   # print(X)
    global df_buffer_x
    temp_X = X
    temp_y = y
    temp_y = temp_y.iloc[0:0]
    temp_X = temp_X.iloc[0:0]
    
    to_add_for_one_element = round(samples_to_add/len(df_buffer_x),0)
    #print("to_add_for_one_element:",to_add_for_one_element)
    min_buf_copy = df_buffer_x
    min_buf_copy = min_buf_copy.drop(['life', 'recall_weight','total_weight'], axis=1)
    #print(len(min_buf_copy))
    for v in range (0,len(min_buf_copy)):
        #print(X.iloc[[v]])
        for c in range(0,int(to_add_for_one_element)):
            #print("adding resample")
            temp_X = temp_X.append(min_buf_copy.iloc[[v]])
            temp_y = temp_y.append({'class': 1}, ignore_index=True)
            #print(len(temp_X))
    #print(temp_X)
    #print("resample from buffer--return rows")
    #print(temp_X)
    #print(temp_y)
    
   # print(len(temp_X))
    #print(len(temp_y))
    return temp_X,temp_y

def remove_zero_life_rows():
    global df_buffer_x
    #print("before:",len(df_buffer_x))  
    df_buffer_x = df_buffer_x[df_buffer_x['total_weight'] >= 0.50]
    #print("after:",len(df_buffer_x))


#performs random oversampling  
def get_weighted_buffer_data(X,y):
    global df_buffer_x
    temp_X = X
    temp_y = y
    temp_y = temp_y.iloc[0:0]
    temp_X = temp_X.iloc[0:0]
    min_buf_copy = df_buffer_x #.loc[df_buffer_x['total_weight'] >= 0.50] 
    min_buf_copy = min_buf_copy.drop(['life', 'recall_weight','total_weight'], axis=1)    
    for v in range (0,len(min_buf_copy)):
            temp_X = temp_X.append(min_buf_copy.iloc[[v]])
            temp_y = temp_y.append({'class': 1}, ignore_index=True)    
    return temp_X,temp_y

#code for fixing clusters in the data
#user class_to_process=1 for minority space -- creates equal size clusters.
#for majority, set class_to_process=0, performs sampling within clusters without making them equal size.
def fix_disjuncts(X,y,samples_to_add,class_to_process,DontAddToBuffer,n_features):
    global disjuncts
    global output_folder
    global disjuncts_threshold
    global arr_ord_cols
    global minority_buffer_x
    global minority_buffer_y
    global w
    global df_buffer_x
    #if minority is less than 5% in the batch 
    #then use minority buffer to do get data.    
    min_min_count = int(w*0.05) #we need atleast 5% samples for clustering
    
     

    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    #print(X)
    if(len(X) < 10):
        return X,y
    
    data_scaled = X
    n_samples, n_features = X.shape
    try:
        data_scaled = preprocessing.scale(X)
    except:
        return X,y
    linked = linkage(data_scaled, 'average')
    last = linked[-10:, 2]
    last_rev = last[::-1]
    idxs = np.arange(1, len(last) + 1)
    accele = np.diff(last, 2)  # 2nd derivative of the distances
    accele_rev = accele[::-1]
    k = accele_rev.argmax() + 2  # if idx 0 is the max of this we want 2 clusters

    model = AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto', connectivity=None, linkage='average', memory=None, n_clusters=k)
    y_hc=model.fit_predict(X)
    aggloclust=model.fit(X)
    (unique, counts) = np.unique(aggloclust.labels_, return_counts=True)
    frequencies = np.asarray((unique, counts)).T
    frequencies = frequencies[np.argsort(frequencies[:, 1])[::-1]]
    arr_cumsum = np.cumsum(frequencies,axis=0)
    cumsum = arr_cumsum[:,1]
    arr_sum_cumsum = np.sum(frequencies,axis=0)
    arr_clusters = []
    sum_cumsum = arr_sum_cumsum[1]
    for i in range(len(arr_cumsum)):
        val = cumsum[i]
        arr_clusters.append(frequencies[i,0])

    
    arr_y_frames  = []
    arr_x_frames  = []
    cluster1_count =0
    df_x1 = []
    df_y1 = []
    
    disjuncts = len(arr_clusters)
    if(disjuncts<disjuncts_threshold):
        return X,y
    
    valid_clusters =0
    for i in range(len(arr_clusters)):
        arr_indexes = ((np.where(y_hc == int(arr_clusters[i]))))
        df_y = y.loc[y.index[arr_indexes]]
        if(len(df_y) > 2):
            valid_clusters += 1
    if(valid_clusters == 0):
        return X,y
    
    each_cluster_addition = int(np.round(samples_to_add / valid_clusters ))  
    for i in range(len(arr_clusters)):
            arr_indexes = ((np.where(y_hc == int(arr_clusters[i]))))
            df_x = X.loc[X.index[arr_indexes]]
            df_y = y.loc[y.index[arr_indexes]]
            
            
            if(len(df_x) <3):
                continue
            
            to_add = 0
            if(i==0):
                cluster1_count = len(df_x)
                to_add = each_cluster_addition
            else:
                to_add = each_cluster_addition
                
                #code to make equal size clusters in minority
                if(class_to_process == 1):
                    current_cluster_count = len(df_x)
                    to_add += (cluster1_count-current_cluster_count)
                   
            df_cov = df_x.cov()
            df_mean = df_x.mean(axis=0)
            try:
                new_samples = np.random.multivariate_normal(df_mean, df_cov,to_add)
            except:
                return X,y
            
            cols = []
            for f in range(1,n_features+1):
                cols.append(str(f))
            df_new_samples = pd.DataFrame(new_samples,columns=cols)
            df_x = pd.concat([df_x.reset_index(drop=True),df_new_samples.reset_index(drop=True)],axis=0)
            df_x = df_x.reset_index(drop=True)
            if(len(arr_ord_cols)>0):
                for h in range (0,len(arr_ord_cols)):
                    ix = arr_ord_cols[h]
                    df_x[str(ix)] = round(df_x[str(ix)]) 
            for j in range(0,to_add):
                df_y = df_y.append(pd.Series([class_to_process],index=['class']),ignore_index = True) 
            
            df_y = df_y.reset_index(drop=True)            
            arr_x_frames.append(df_x)
            arr_y_frames.append(df_y)        
    
    np.set_printoptions(threshold=np.inf)            
    modified_X = pd.concat(arr_x_frames)
    modified_Y = pd.concat(arr_y_frames)
    
    modified_X = modified_X.reset_index(drop=True)
    modified_Y = modified_Y.reset_index(drop=True)
    #print("**********</Fix Disjuncts End>************")
    #print(modified_X.shape)
    return modified_X,modified_Y    



def icrc(X_maj,X_min,y_maj,y_min,n_features,overlap_val,fullFit,ups,downs):
    
    
    
    global f1_overlap_value
    global overlap_threshold_f1
    global imbalance_ratio_threshold
    ##################### Update Imbalance status of current window
    is_imbalance = False
    ir = 0
    if(ups ==0 or downs ==0):
        ir = imbalance_ratio_threshold + 1
    elif(ups > downs):
        ir = ups/downs
    elif(ups < downs):
        ir = downs/ups
    elif(ups == downs):
        ir = 1

    if(ir>=imbalance_ratio_threshold):
        is_imbalance = True
    ########################################

    #is_imbalance = False
    if(is_imbalance == False):
        frames = [X_min,X_maj]
        modified_X = pd.concat(frames)
        frames = [y_min,y_maj]
        modified_Y = pd.concat(frames).replace('UP',1).replace('DOWN',0) 
        return modified_X.to_numpy(),modified_Y.to_numpy().reshape(-1)

    if(is_imbalance==True):
        is_overlap = False
        fitted_already = False

        
        modified_Xmin,modified_Ymin = fix_disjuncts(X_min,y_min,(len(X_maj)-len(X_min)),1,fullFit,n_features)
        modified_Xmaj= X_maj
        modified_Ymaj = y_maj
        f1_overlap_value = overlap_val
        if (f1_overlap_value > overlap_threshold_f1): 
            #print("overlap true")
            is_overlap =  True
        if is_overlap:
            modified_Xmaj = modified_Xmaj.reset_index(drop=True)
            modified_Xmin = modified_Xmin.reset_index(drop=True)
            idx_maj = len(modified_Xmaj)

            
            all_data = pd.concat([modified_Xmaj,modified_Xmin])

            a = pdist(all_data,metric='euclidean')

            t_l = len(all_data)

            d_1 = linkage(a)
            
            d_2 = pd.DataFrame(d_1,columns=['i1','i2','i3','i4'])
            filtered_values = np.where( ((d_2["i1"] < idx_maj) & (d_2["i2"] > idx_maj) & (d_2["i2"] <t_l)  ) | ((d_2["i1"] > idx_maj) & (d_2["i1"] < t_l ) & (d_2["i2"] < idx_maj) ) )

            filtered =  d_2.loc[filtered_values]
            arr_indexes = filtered.iloc[:,0].to_numpy() 

            to_remove = int(len(arr_indexes)*disjunct_threshold_perc_maj)        
            drop_these = (arr_indexes[0:to_remove])
            modified_Xmaj.drop(index=drop_these,inplace=True)
            modified_Ymaj = modified_Ymaj.iloc[to_remove:]


        if(len(modified_Xmin) > len(modified_Xmaj)):
            modified_Xmaj,modified_Ymaj = fix_disjuncts(modified_Xmaj,modified_Ymaj,(len(modified_Xmin)-len(modified_Xmaj)),0,fullFit,n_features)

        frames = [modified_Xmin,modified_Xmaj]
        modified_X = pd.concat(frames)
        frames = [modified_Ymin,modified_Ymaj]
        modified_Y = pd.concat(frames).replace('UP',1).replace('DOWN',0) 
        
    return modified_X.to_numpy(),modified_Y.to_numpy().reshape(-1)




class DetectorClassifier(BaseEstimator):
    def __init__(self, clf, classes):
        if not hasattr(clf, "partial_fit"):
            raise TypeError("Choose incremental classifier")
        self.clf = clf
        classes = [0,1]
        self.classes = classes
        self.change_detected = 0

    def fit(self, X, y):
        #self.clf = clone(self.clf)
        self.clf.fit(X, y)
        return self

    def partial_fit(self,modified_X,modified_Y,fullFit):
            if(fullFit):
               # print("^^^^Full Fit^^^")    
                self.clf = self.clf.fit(modified_X, modified_Y, classes=self.classes)            
            else:
               # print("^^^^Partial Fit^^^")
                self.clf = self.clf.partial_fit(modified_X, modified_Y, classes=self.classes)

            return self
    
    ##### Prdict Function        
    def predict(self, X):
        return np.asarray(self.clf.predict(X), dtype=np.int64).ravel()
    
    ##### Score Function
    def score(self, X,y):
        return self.clf.score(X,y)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X) 
    
def bin_counts(data_length):
    #Sturges' Rule
    #num_bins = 1 + int(np.log2(data_length))
    
    #Sq. Root Rule
    num_bins = int(sqrt(data_length))
    return num_bins



def doanes_formula(data):
    n = len(data)
    
    #return bin_counts(n)
    
    # Calculate skewness and standard error of skewness
    skewness = 3 * (np.mean(data) - np.median(data)) / np.std(data)
    se_skewness = np.sqrt(6 * (n - 2) / ((n + 1) * (n + 3)))
    
    # Calculate number of bins using Doane's formula
    if(skewness == 0):
        num_bins = 1 + log2(n)
    else:
        
        num_bins = 1 + log2(n) + log2(1 + abs(skewness) / se_skewness)
    print(num_bins)
    return int(num_bins)


    

def sigmoid_decay(t):
    global initial_value
    global final_value
    global time_constant
    return initial_value + (final_value - initial_value) / (1 + np.exp(-(t - time_constant) / time_constant))
   
    
def sigmoid_decay_ensemble_weight(t):
    global initial_value
    global final_value
    global time_constant_ensemble_weight
    return initial_value + (final_value - initial_value) / (1 + np.exp(-(t - time_constant_ensemble_weight) / time_constant_ensemble_weight))
   
def calculate_cohens_kappa(actual, predicted):
    # Confusion Matrix
    tp = sum((a == 1) and (p == 1) for a, p in zip(actual, predicted))
    tn = sum((a == 0) and (p == 0) for a, p in zip(actual, predicted))
    fp = sum((a == 0) and (p == 1) for a, p in zip(actual, predicted))
    fn = sum((a == 1) and (p == 0) for a, p in zip(actual, predicted))
    
    # Observed agreement
    observed_agreement = (tp + tn) / len(actual)
    
    # Calculate expected agreement
    expected_agreement = ((tp + fn) * (tp + fp) + (tn + fp) * (tn + fn)) / (len(actual) ** 2)
    
    # Calculate Kappa
    kappa = (observed_agreement - expected_agreement) / (1 - expected_agreement)
    
    return kappa
       
def prequential(window_size,Xy,X, y, clf,n_features,overlap_threshold,imbalance_ratio_threshold,disjunct_threshold_perc_maj,file_out_prefix, n_train=1):
    
    """Prequential Evaluation: instances are first used to test, and then to train
    :return the label predictions for each test instance, and the associated running time
    """
    first_weights_flag = False
    
    arr_clf = []
    arr_clf_temp_to_use = []
    arr_weights = []
    arr_recall0_cw = []
    arr_recall1_cw = []
    
    a= 0.00001
    
    start = timer()

    global arr_elapsedtime
    global restart_threshold
    global arr_recall_0
    global arr_recall_1
    global output_folder
    global maj_weight
    global min_weight
    global ens_weight
    global ensemble_pool_size    
    global pruned_ensemble_size
    global rl_pruned_ids
    global preq_Qtable
    global col_means
    global imputed_from_arm
    global total_miss_count
    global all_rules_df
    
    global all_models_df
    global arr_all_gmeans
    global arr_all_recall0
    global arr_all_recall1
    global str_settings
    global arr_all_perc_replaced
    
    global col_means
    global imputed_missing_data
    
    global df_weights_history
    
    weights_pruned = []
    arr_actual_win = []
    arr_pred_win = []
    arr_roc = []
    arr_divs = []
    ensemble_tp = 0
    ensemble_fp =0
    ensemble_fn = 0
    ensemble_tn = 0
    
    
    ensemble_correct =0
    zeroClassCounter = 0
    oneClassCounter = 0
    current_correct_0 = 0
    current_correct_1 = 0
    
    current_correct_ensemble_0 = 0
    current_correct_ensemble_1 = 0
    
    predictions = ""    
    row_num = y.shape[0]   
   
    # Split an init batch
    X_init = X[0:n_train]
    y_init = y[0:n_train]
    
    # Used for training and evaluation
    X_train = X
    y_train = y
    
    Xy_train = Xy
    
        
    ensemble_preds = np.zeros(row_num)
    current_correct_0_cw = np.zeros(ensemble_pool_size)
    current_correct_1_cw = np.zeros(ensemble_pool_size)
    recall_0_cw = np.zeros(ensemble_pool_size)
    recall_1_cw = np.zeros(ensemble_pool_size)
    
    rows_to_remove = np.any(np.isnan(X_init) | (X_init == None), axis=1)
    X_init = X_init[~rows_to_remove]

    col_means = np.mean(X_init,axis=0)
    

    indices_top_n = np.argsort(y_init)[-len(X_init):]

    # Select the top n elements based on indices
    top_n_elements = y_init[indices_top_n]
    for g in range (0,ensemble_pool_size):
        #Create a classifier and add it to the pool
        obj_clf = DetectorClassifier(HoeffdingTreeClassifier(), [0,1])
        
        #print(top_n_elements)
        obj_clf = obj_clf.fit(X_init,top_n_elements)
        arr_clf.append(obj_clf)
        arr_weights.append(1)
        df_weights_history = df_weights_history.append({'i':g},ignore_index = True)
        
    ##print(df_weights_history)
    
    #sys.exit()
    current_w = 0
    window_counter = 0
    strProcessing = ""
    correct_predictions = 0

    strout = ""
    strgmean = ""

    oneClassCounter_CWindow = 0
    zeroClassCounter_CWindow = 0
    arr_cw_ens_preds = []
    current_correct_0_cw_est = 0
    current_correct_1_cw_est = 0
    window_recall_0 = 0
    window_recall_1 = 0
        
    for i in range(0, row_num):
        #print(X[i, :])
        #sys.exit()
        current_w += 1
        start_time = perf_counter()
        arr_preds = []
        arr_temp_preds = []
        row_process = impute_row(X_train[i, :],False)
        for n in range(0,len(arr_clf)):
            clf = arr_clf[n]
            for_pred = row_process.reshape(1, -1)
            try:
                prediction_clf = clf.predict(for_pred)
                arr_preds.append(prediction_clf[0])
                arr_temp_preds.append(prediction_clf[0])
            except:
                print("predict:",for_pred)
                sys.exit()
        arr_cw_ens_preds.append(arr_temp_preds)   
        strout += "------------\n"
        estimators_arr = []
        for n in range(0,len(arr_clf)):
            estimators_arr.append(("HT-" + str(n), arr_clf[n]))
        
       
        df_estimators_arr = pd.DataFrame(arr_clf)
        
        
        arr_clf_temp_to_use = []
        estimators_arr_temp = []
        arr_weights_temp = []

        ## Ensemble Pruning
        for h in range(0,len(rl_pruned_ids)) :
            try:
                indexx = rl_pruned_ids[h].astype(int)
            except:
                indexx = rl_pruned_ids[h]
            #print("using classifier on index:",indexx)
            arr_clf_temp_to_use.append(arr_clf[indexx])
            estimators_arr_temp.append(("HT-" + str(indexx), arr_clf[indexx]))
            #arr_weights_temp.append(arr_weights[indexx]) #old 
            arr_weights_temp.append(weights_pruned[h]) #received from clustering of classifiers
        
        if(len(arr_clf_temp_to_use) == 0):
            for h in range(0,len(arr_clf)) :
                arr_clf_temp_to_use.append(arr_clf[h])
                estimators_arr_temp.append(("HT-" + str(h), arr_clf[h]))
                arr_weights_temp.append(1)
        pred = 0
        try:
            eclf = VotingClassifier(estimators = estimators_arr_temp, voting='soft',weights=arr_weights_temp) 
            eclf.estimators_ = arr_clf_temp_to_use
            eclf.le_ = LabelEncoder().fit(np.unique([0, 1]))
            for_pred = row_process.reshape(1, -1)
            #print("predict:",for_pred)
            pred_proba = eclf.predict_proba(for_pred)
            pred = eclf.predict(for_pred)
            arr_tempo = pred_proba[0]
            arr_pred_win.append(pred)
        except:
            eclf = VotingClassifier(estimators = estimators_arr_temp, voting='hard',weights=arr_weights_temp) 
            eclf.estimators_ = arr_clf_temp_to_use
            for_pred = row_process.reshape(1, -1)
            #print("predict:",for_pred)
            eclf.le_ = LabelEncoder().fit(np.unique([0, 1]))
            pred = eclf.predict(for_pred)
            if(pred==0):
                arr_pred_win.append(0)
            if(pred==1):
                arr_pred_win.append(1)  
        ensemble_preds[i] = pred
        counter = i+1
        lambda_0 = maj_weight#Majority
        lambda_1 = min_weight #Minority
        
        if y_train[i] == 0:
            zeroClassCounter += 1
            zeroClassCounter_CWindow += 1
        else: 
            oneClassCounter += 1
            oneClassCounter_CWindow += 1
        correct_found = 0

        for n in range(0,len(arr_clf)):
            if(y_train[i] == 0 and arr_preds[n] ==0):
                current_correct_0_cw[n] = current_correct_0_cw[n]+1
            if(y_train[i] == 1 and arr_preds[n] ==1):    
                current_correct_1_cw[n] = current_correct_1_cw[n]+1
        
        correct_found = 0  
        y_actual = 0
        y_pred = 0
        if(y_train[i] ==0 and pred ==0):
            correct_found +=1
            current_correct_ensemble_0 += 1
            current_correct_0_cw_est += 1
            ensemble_tn += 1
            y_actual = 0
            y_pred = 0
        if(y_train[i] ==1 and pred == 1):
            correct_found +=1
            current_correct_ensemble_1 += 1
            current_correct_1_cw_est += 1
            ensemble_tp += 1
            y_actual = 1
            y_pred = 1
        if(y_train[i] ==1 and pred ==0):
            ensemble_fn += 1
            y_actual = 1
            y_pred = 0
        if(y_train[i] ==0 and pred == 1):
            ensemble_fp += 1
            y_actual = 0
            y_pred = 1
        if(correct_found ==1):
            ensemble_correct += 1
        arr_actual_win.append(y_actual)
        #when the batch is ready or an in complete batch at the end
        if( ( current_w % window_size == 0)  or (current_w != window_size and i == (row_num - w -1)) ):

            window_counter += 1
            window_recall_0 = 0
            window_recall_1 = 0
            if zeroClassCounter_CWindow != 0:
                window_recall_0 =  current_correct_0_cw_est / zeroClassCounter_CWindow
            if oneClassCounter_CWindow != 0:
                window_recall_1 =  current_correct_1_cw_est / oneClassCounter_CWindow
            
            window_gmean = sqrt(window_recall_0*window_recall_1)            
            
            update_recall_weight( (lambda_0*window_recall_0) + (lambda_1*window_recall_1))

            kappa = cohen_kappa_score(arr_actual_win, arr_pred_win)
            arr_kappa.append(kappa)
            
            arr_gmean.append(window_gmean*100)
            arr_recall_0.append(window_recall_0*100)
            arr_recall_1.append(window_recall_1*100)
            
            accuracy = (ensemble_tp+ensemble_tn)/(zeroClassCounter_CWindow+oneClassCounter_CWindow)
    
            arr_actual_win=[]
            arr_pred_win = []
                       
            arr_accuracy.append(accuracy)
            for n in range(0,len(arr_clf)):                    
                if zeroClassCounter_CWindow != 0:
                    recall_0_cw[n] =  current_correct_0_cw[n] / zeroClassCounter_CWindow                    
                else:
                    recall_0_cw[n] =  1
                    
            for n in range(0,len(arr_clf)):
                    if oneClassCounter_CWindow != 0:
                        recall_1_cw[n] =  current_correct_1_cw[n] / oneClassCounter_CWindow
                    else:
                        recall_1_cw[n] = 1

                
            for n in range(0,len(arr_clf)):
                weight = (lambda_0 * recall_0_cw[n]) + (lambda_1*recall_1_cw[n])                
                arr_weights[n] = weight
                
                row_to_update = df_weights_history[df_weights_history['i']==n]
                for r in range (win_weight_count,1,-1):
                    df_weights_history.loc[df_weights_history['i'] ==n, str(r)] = df_weights_history.loc[df_weights_history['i'] ==n, str(r-1)]   
                df_weights_history.loc[df_weights_history['i'] ==n, '1'] = weight  

            

            arr_weights = normalize(arr_weights)
            
            oneClassCounter_CWindow = 0
            zeroClassCounter_CWindow = 0  
            current_correct_0_cw_est = 0
            current_correct_1_cw_est = 0
            ensemble_tn = 0
            ensemble_tp = 0
            ensemble_fp = 0
            ensemble_fn = 0
            
            for n in range(0,len(arr_clf)):
                current_correct_0_cw[n]=0
                current_correct_1_cw[n] = 0
            
            
            x_preq = X_train[i-(window_size-1):i+1]
            y_preq = y_train[i-(window_size-1):i+1]

            Xy_preq = Xy_train[i-(window_size-1):i+1]
            ups =  np.count_nonzero(y_preq == 1)
            downs = np.count_nonzero(y_preq == 0)         

            
            arr_clf_temp = []

            ####################### <Overlap Detection> ############################################
            #Check overlaps where and send its status to partialfit
            cols_str = []
            for f in range(1,n_features+1):
                cols_str.append(str(f))
            cols_str.append('class')
            Xy_d = pd.DataFrame(Xy_preq, columns = cols_str)

            Xy_min = Xy_d[(Xy_d['class'] == 1)]
            Xy_maj = Xy_d[(Xy_d['class'] == 0)]

            cols = []
            for f in range(0,n_features):
                cols.append(f)
            cols_class = [n_features]
            X_min = Xy_min[Xy_min.columns[cols]]
            X_maj = Xy_maj[Xy_maj.columns[cols]]
            y_min = Xy_min[Xy_min.columns[cols_class]]
            y_maj = Xy_maj[Xy_maj.columns[cols_class]]
            
            frames = [X_min,X_maj]
            modified_X = pd.concat(frames)
            frames = [y_min,y_maj]
            modified_Y = pd.concat(frames)

            cols_num = []            
            for p in range(0,n_features):
                cols_num.append(p)
            m_X = modified_X
            m_Y = modified_Y            
            m_X.columns = cols_num
            m_Y.columns = [0]  
            
            missing_data = X_min[X_min.isnull().any(axis=1)]
            
            
            prev_col_means = col_means
            col_means = X_min.mean(skipna=True).values 
            contains_nan_or_none = np.isnan(col_means).any()
            #print(contains_nan_or_none)
            if(contains_nan_or_none):
                col_means = prev_col_means
            do_regress(X_min,n_features,window_counter)
            imputed_resamples = missing_data.copy()
            imputed_resamples.drop(imputed_resamples.index).reset_index(drop=True)
            
            y_min_resampled = y_min.copy()
            y_min_resampled = y_min_resampled.drop(y_min_resampled.index).reset_index(drop=True)
            
            
            if(window_counter == 10000):
                print(missing_data)
                print(x_preq)
                #sys.exit()
            
            for x in range (0,len(missing_data)):
                a= impute_row(missing_data.iloc[x],True)
                missing_data.iloc[x] = a
                imputed_missing_data.append(missing_data.iloc[x])
                    
               
            X_min_non_missing = X_min.dropna()
            y_min_non_missing = y_min.head(len(X_min_non_missing))
            
            frames = [X_min_non_missing,missing_data]
            X_min = pd.concat(frames)       
            #print("x size b4:",len(X_min))
            for t in range (0, len(X_min)):
                new_row = {'class': 1} 
                y_min_resampled.loc[len(y_min_resampled)] = new_row
            
            if(len(X_maj) ==0 or len(X_min_non_missing) == 0):
                 overlap_val = 0                   
            else:
                overlap_val, fishers = detect_overlap(X_min_non_missing, X_maj,n_features)   
            modified_x,modified_y = icrc(X_maj,X_min,y_maj,y_min_resampled,n_features,overlap_val,False,ups,downs)
                   
            
            arr_temp_clfs = []
            ## Partial Fit the previous classifiers
            for n in range(0,len(arr_clf)):
                clf = arr_clf[n]
                clf = clf.partial_fit(modified_x,modified_y,False)
                arr_temp_clfs.append(clf)
             
            
            #################################### Commented out code that removes the minimum preto based member ###############
            arr_clf = arr_temp_clfs
            remove_index = 0
            if(len(arr_clf) == ensemble_pool_size):
                least_weight = arr_weights[0]
                for q in range (1,len(arr_weights)-1):
                    w_q = arr_weights[q]
                    if(w_q < least_weight):
                        remove_index = q
                #print("removing:",remove_index)
                arr_clf = np.delete(arr_clf,remove_index)
                arr_weights = np.delete(arr_weights,remove_index)
            
            for o in range(remove_index,ensemble_pool_size-1):
                #print("o:",o)
                #print(df_weights_history.loc[df_weights_history['i'] ==o, str(1)])
                
                for r in range (1,win_weight_count+1):
                    df_weights_history.loc[df_weights_history['i'] ==o, str(r)] = df_weights_history.loc[df_weights_history['i'] ==(o+1), str(r)].iloc[0]  
                    
            df_weights_history.loc[df_weights_history['i'] == (ensemble_pool_size-1), str('1')] = 1  
            for r in range (2,win_weight_count+1):
                df_weights_history.loc[df_weights_history['i'] == (ensemble_pool_size-1), str(r)] = np.nan
            
            
            arr_weights_decayed = []
            for o in range(0,ensemble_pool_size): 
                #print("---\n\n next member:")
                weight = 0
                for r in range (1,win_weight_count+1):
                    val = df_weights_history.loc[df_weights_history['i'] == (o), str(r)].iloc[0]
                    if( np.isnan(val) == False ) :
                        #print("actual val:",val)
                        decayed = sigmoid_decay_ensemble_weight(r) * val
                        weight += decayed
                arr_weights_decayed.append(weight)
            ######################### add new classifier to the pool after training it with the current window.#####################
            
            obj_clf1 = DetectorClassifier(HoeffdingTreeClassifier(), [0,1])
            clf_temp =obj_clf1.partial_fit(modified_x,modified_y,True)
 
            arr_weights = np.append(arr_weights,1)
            arr_clf = np.append(arr_clf,clf_temp)

            
            '''### diversity work '''
            arr_temp = np.vstack(arr_cw_ens_preds) 
            arr_temp = np.delete(arr_temp,remove_index,axis=1)
            arr_div_q = []
            arr_div_df = []            
            arr_div_da = []            
            
            for m in range(0,len(arr_clf)-1):
                dq_sum = 0
                ddf_sum = 0
                dda_sum = 0
                
                for n in range(0,len(arr_clf)-1):
                    if(m == n):
                        continue
                    ens_preds_a = arr_temp[:,m]
                    ens_preds_b = arr_temp[:,n]
                    dq= 1
                    ddf = 1
                    dda = 1
                    try:
                        
                        dq,ddf,dda = cal_diversities(y_preq, ens_preds_a, ens_preds_b)
                    except: 
                        
                        ddq,ddf,dda = 1,1,1
                    dq_sum += dq
                    ddf_sum += ddf
                    dda_sum += dda
                    
                avg_dq = 0    
                avg_ddf = 0
                avg_dda = 0    
                
                if(len(arr_clf) != 1):
                    #average over all 
                    avg_dq = (dq_sum)/(len(arr_clf)-2)      
                    avg_ddf = (ddf_sum)/(len(arr_clf)-2)                  
                    avg_dda = (dda_sum)/(len(arr_clf)-2)                  
                    
                    
                arr_div_q.append(avg_dq)
                arr_div_df.append(avg_ddf)
                arr_div_da.append(avg_dda)                
                
            arr_cw_ens_preds = [] 
            rl_pruned_ids = []   
            #Apply clustering on diversities
            df_clfs = pd.DataFrame( columns =['index','dq','ddf','dda','weight','final_weight'])
            for v in range(0,len(arr_div_q)):

                df_clfs = df_clfs.append({'index':v,'dq' : arr_div_q[v], 'ddf' : arr_div_df[v],'dda' : arr_div_da[v],'weight' : arr_weights_decayed[v],'final_weight' : 1},ignore_index = True)

            df_clfs = df_clfs.append({'index':len(arr_div_q),'dq' : 0.5, 'ddf' : 0.5,'dda' : 0.5,'weight' : 0.5,'final_weight' : 0.5},ignore_index = True)
            
            rl_pruned_ids,weights_pruned = cluster_classifiers(df_clfs)
            
            '''### diversity work ends here '''
            
            
             global pruned_ensemble_size
             
            ##########################################################################################################        
            global model_interval
            current_w = 0
            prev_win_x = x_preq
            prev_win_y =  y_preq
            
            end = timer()
            elapsed_time = end - start
            arr_elapsedtime.append(elapsed_time) 
            print(str(window_counter) + "-"+str(np.mean(arr_gmean)) + "  --- " + str(np.mean(arr_kappa)) + "  --- " + str(elapsed_time))
                #sys.exit()
            #   sys.exit()
            
    strout += "\n total corrects:" + str(ensemble_correct)
    strout += "\n zero+one:" + str(zeroClassCounter+oneClassCounter)
    np.savetxt(output_folder + file_out_prefix  +'.csv',  np.c_[arr_recall_0,arr_recall_1,arr_gmean,arr_elapsedtime,arr_accuracy,arr_kappa], delimiter=',',fmt='%1.2f')
    return  np.mean(arr_recall_0), np.mean(arr_recall_1),np.mean(arr_gmean)


def do_regress(X_min,n_features,window_counter):
    epochs =1
    global all_models_df
    global all_models
    global X_min_for_regress
    global time_constant
    global arr_rules_history
    global model_interval
    global w
    if X_min_for_regress is None:
        X_min_for_regress = X_min.copy()
        X_min_for_regress = X_min_for_regress.drop(X_min_for_regress.index).reset_index(drop=True) 
        X_min_for_regress = X_min_for_regress.append(X_min, ignore_index=True) 
    else:
        X_min_for_regress = X_min_for_regress.append(X_min, ignore_index=True)    
    

    if window_counter < model_interval:
        return
        
        
    if window_counter % model_interval != 0:
        return
    
    for index, row in all_models_df.iterrows():
        all_models_df.at[index,'weight'] = round(sigmoid_decay((window_counter - row['window'])/model_interval) * row['weight'] , 4)
    all_models_df = all_models_df[all_models_df['weight'] > 0.00]
    #print("total rows after removal:",len(all_models_df))   

    for ep in range(0,epochs):
        #generate a random number atleast more than 2
        if(n_features > 12):
            k = random.randint(2, 10)        
        else:
            k = random.randint(2, n_features)        
        epochs_1 = 1
        for ep1 in range (0,epochs_1):
            #print("inner_episode")
            n_f = random.sample(range(0,n_features),k)
            
            X_min_c = X_min_for_regress.iloc[:,n_f]
            X_min_c = X_min_c.dropna()  
            
            if(len(X_min_c)<4):
                continue
            #print("len:",X_min_c)
            
            for f in range (0,len(n_f)):
                ind_list = n_f.copy()
                ind_list.pop(f)
                ind_list = [element + 1 for element in ind_list]
                dep = n_f[f]+1
                ind_list = list(map(str, ind_list))
                X = X_min_c[ind_list]
                y = X_min_c[str(dep)]
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
                

                # Create and fit a linear regression model
                model = LinearRegression()
                model.fit(X, y)

                # Make predictions on the test set
                y_pred = model.predict(X_test)
                
                #y_test = y
                y_test = y_test.values
                # Evaluate the model
                
                mse = mean_squared_error(y_test, y_pred)
                rmse = np.sqrt(mse)
                r2 = r2_score(y_test, y_pred)
                
                weight = (0.5 * r2) + 0.5 * (len(X)/model_interval)
                all_models_df["rmse"] = all_models_df["rmse"].astype(float)
                all_models_df["dep"] = all_models_df["dep"].astype(int)
                model_id = str(uuid.uuid4())
                all_models_df = all_models_df.append({'model_id' : model_id,'ind' : ','.join(ind_list), 'dep' : str(dep), 
                                            'mse' : str(mse), 'rmse' : str(rmse), 'r2' : str(r2), 'window' : window_counter, 'life' : 1, 
                                                       'coef' : model.coef_, 'intercept' : model.intercept_, 'weight' : weight},ignore_index = True)
                all_models[model_id] = model
                    
                all_models_df["rmse"] = all_models_df["rmse"].astype(float)
                all_models_df["dep"] = all_models_df["dep"].astype(int)
    X_min_for_regress = None
    arr_rules_history.append(len(all_models_df))
    
    
    
    
     
    
def cluster_classifiers(df_clfs):
    global diversity_lambda
    global weight_lambda

    data_scaled = df_clfs
    data_scaled = preprocessing.scale(df_clfs)
    max_weight_epochs = 0
    indexes_for_cluster = []
    for a in range (0,1):
        #k = random.randint(3, 6)
        #print("in pruning:",df_clfs)
        features = df_clfs.drop(columns=['index','weight','final_weight'], axis=1)
        data_scaled = features
        
        try:
            data_scaled = preprocessing.scale(features)
            linked = linkage(data_scaled, 'average')
            last = linked[-10:, 2]
            last_rev = last[::-1]
            idxs = np.arange(1, len(last) + 1)
            accele = np.diff(last, 2)  # 2nd derivative of the distances
            accele_rev = accele[::-1]
            k = accele_rev.argmax() + 2  # if idx 0 is the max of this we want 2 clusters
            
        except:
            print("k is random")
            k = random.randint(3, 6)

        
        #print("k:",k)
        
        model = AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto', connectivity=None, linkage='average', memory=None, n_clusters=k)
        df_clfs['cluster'] = model.fit_predict(features)
       
        
        # Step 1: Calculate the average of dq, ddf, and dda
        average_values_div = df_clfs[[ 'dq','ddf', 'dda']].mean(axis=1)

        # Step 2: Multiply the average by 0.5
        average_values_div *= diversity_lambda

        #print("divresity averages:",average_values_div)
        
        # Step 3: Multiply weight by 0.5
        df_clfs['weight'] *= weight_lambda
        
        
        
        # Step 4: Sum the results and assign to final_weight
        df_clfs['final_weight'] = average_values_div + df_clfs['weight']
        

        # Assuming your DataFrame is named df
        result = df_clfs.groupby('cluster')['final_weight'].mean().reset_index()
        # Display the result
        #print("result:",result)
        
        max_cluster = result.loc[result['final_weight'].idxmax(), 'cluster']

        weight_found = result.loc[max_cluster,'final_weight']
        if (weight_found > max_weight_epochs):
            max_weight_epochs = weight_found
            indexes_for_cluster = df_clfs[df_clfs['cluster'] == max_cluster].index.tolist()
            indexes_for_cluster = np.array(indexes_for_cluster)
            weights = df_clfs.loc[indexes_for_cluster,'final_weight']
        
    return np.array(indexes_for_cluster),weights.values
        
    
def read_data(filename):
    df = pd.read_csv(filename,header=None)
    data = df.values
    return data, data[:, :-1], data[:, -1]

def delete_files_from_folder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))


# Function to convert tuples to strings and remove parentheses and trailing commas
def convert_tuple_to_string(tup):
    return str(tup).lstrip('(').rstrip(')').rstrip(',')



def custom_compare(str1, str2):
    num_part1, rest1 = ''.join(filter(str.isdigit, str1)), ''.join(filter(lambda x: not x.isdigit(), str1))
    num_part2, rest2 = ''.join(filter(str.isdigit, str2)), ''.join(filter(lambda x: not x.isdigit(), str2))

    # Compare numerical parts
    if int(num_part1) != int(num_part2):
        return False

    # Compare remaining parts lexicographically
    return rest1 == rest2

def impute_row(missing_row,training_impute):
    global col_means
    global total_miss_count
    global imputed_from_arm
    global arr_ord_cols
    global all_models_df
    global all_models
    # Check for NaN values
    contains_nan = np.isnan(missing_row).any()
    global arr_ord_cols
    
    if contains_nan:  
        
        if(np.isnan(col_means).any()):
            print("col means (during impute null):",col_means)
        if(len(all_models_df) == 0):
            for i in range(0,len(missing_row)):
                if(np.isnan(missing_row[i])):
                    val = col_means[i]  
                    if(i in arr_ord_cols):
                        val = int(val)
                    missing_row[i] = val                  
                    if training_impute:
                        #imputed_from_arm += 1
                        total_miss_count += 1
                        
            return missing_row
        
      
        missing_row = missing_row.tolist()
        row_df = pd.DataFrame({'data': missing_row})
        row_df_org = row_df
        l = -1
        for index, row in row_df_org.iterrows():            
            l+=1
            if(pd.isnull(row[0])):
                #print("index is nan:",index)
                if training_impute:
                    #imputed_from_arm += 1
                    total_miss_count += 1
                
                #get non nan columns of this row
                str_list_non_null_missing_row = ""
                for index1, row1 in row_df_org.iterrows():
                    if( pd.isnull(row1[0]) == False):
                        str_list_non_null_missing_row += str(index1) + ","
                
                pred_arr = []
                weights_arr = []
                pred_count = 0
                    
                #get all models fit for this feature
                df_models = all_models_df[all_models_df['dep'] == l+1]
                for index_model, row_model in df_models.iterrows(): 
                    #print(row_model)
                    ind_list = row_model['ind']
                    coef = row_model['coef']
                    intercept = row_model['intercept']
                    ind_list_arr = ind_list.split(",")
                    rmse = row_model['rmse']
                    rtwo = float(row_model['weight'])
                    #get the model of this ID
                    model = all_models[row_model['model_id']]
                    flag = True
                    pred = 0
                    for h in range (0, len(ind_list_arr)):
                        ind_int_val = int(ind_list_arr[h])
                        ind_str_val = str(ind_int_val - 1)
                        if (ind_str_val in str_list_non_null_missing_row):
                            cof = coef[h]
                            feature_val = str(row_df_org.iloc[ind_int_val-1,0])
                            if(np.isnan(row_df_org.iloc[ind_int_val-1,0])):                            
                                flag = False
                            else:
                                v = coef[h] * row_df_org.iloc[ind_int_val-1,0]
                                pred += v 
                            #continue
                        else:
                            flag = False
                        
                    if(flag):            
                        pred += intercept
                        pred_arr.append(pred)
                        weights_arr.append(rtwo)
                    
                if(len(pred_arr) > 0):
                    pred = 0
                    try:
                        pred = np.average(pred_arr,axis=0,weights=weights_arr)
                        #pred = np.average(pred_arr,axis=0 )
                    except:
                        print("norm error")
                        pred = np.average(pred_arr,axis=0)
                    if(l+1 in arr_ord_cols):
                        pred = int(pred)
                    row_df_org.at[l,'data'] = pred 
                    if training_impute:
                        imputed_from_arm += 1

                if(len(pred_arr) == 0):
                    # Set the new value in the DataFrame
                    pred = col_means[l] 
                    if(l+1 in arr_ord_cols):
                        pred = int(pred)
                    row_df_org.at[l,'data'] = pred 
                  
        a0 = row_df_org.iloc[:,0].values
        return a0
    else:
        return missing_row
    
    
    
    
#*********************************  Main ******************************** #

global imbalance_threshold
global arr_recall_0
global arr_recall_1
global arr_gmean
global arr_accuracy
global arr_kappa
global restart
global restart_threshold
global f1_overlap_value
global maj_weight
global min_weight
global ens_weight
global arr_ord_cols
global preq_Qtable
global restart_threshold
global min_life_max
global min_life_decay_factor
global output_folder
global stream_folder

global ensemble_pool_size
global pruned_ensemble_size
global rl_pruned_ids
global learning_rate
global decay_factor
global minority_buffer_x
global minority_buffer_y
global df_buffer_x
global rl_each_step_episodes
global arr_elapsedtime

global col_means
global all_models_df
global all_models


global total_miss_count
global imputed_from_arm
global rules_decay_factor
global diversity_lambda
global weight_lambda

global min_support
global min_confidence
global arr_rules_history
global initial_value
global final_value
global time_constant

global arr_all_gmeans
global arr_all_recall0
global arr_all_recall1
global arr_all_perc_replaced

global str_settings
global df_weights_history
global win_weight_count


arr_all_gmeans = []
arr_all_recall0 = []
arr_all_recall1 = []
arr_all_perc_replaced = []
global imputed_missing_data

global X_min_for_regress
global model_interval

iterations =6
for k in range (4,iterations):
    arr_elapsedtime = []
    arr_rules_history = []
    df_buffer_x = pd.DataFrame()
    f1_overlap_value = 0

    cwd = os.getcwd()
    cwd = "/home/temp"
    #cwd = "D:\\usman-data"
    output_folder = cwd +  '/output/DRDVEN-V1/main4/'
    stream_folder = cwd +  '/streams/mvi-v1/csvs/missing-real/'

    imputed_missing_data = []

    ensemble_pool_size = 15 #ensemble pool size
    pruned_ensemble_size =8 #pruned ensemble size


    n_train = 400 #no of instances to train the model (offline)
    w =400 #batch size
    overlap_threshold_f1 = 0.0 #overlap threshold
    imbalance_ratio_threshold =1.2 #IR threshold
    disjuncts_threshold =2 #MinD Threshold
    disjunct_threshold_perc_maj = 1.0 #overlap removal threshold
    maj_weight = 0.1 #lambda_0 majority weight in ensemble
    min_weight = 0.9  #Lambda_1 minority weight in ensemble
    min_life_max = 0.99 #minoirty instance life max value
    min_life_decay_factor = 0.5 #minority instance life decay factor

    diversity_lambda = 0
    weight_lambda = 1
    
    #sigmoid delay
    initial_value = 1.0  # Initial value
    final_value = 0.0  # Final value
    time_constant = 0.75 # Time constant controlling the rate of decay for linear models
    time_constant_ensemble_weight =0.75 #ime constant controlling the rate of decay for ensemble weight
    
    model_interval = 5
    win_weight_count = 10
    
    arr_ord_cols= []
    #arr_ord_cols = [5,6,7,8,9,10] #1-based index  GMSC
    #arr_ord_cols = [13,14,15,16,17,18,19,20,21] #1-based index  IJCNN1
    #arr_ord_cols = [10,11,12] #1-based index  LOAN
    #arr_ord_cols = [1,2,3,4,5,6,7,8,9,10] #1-based index  poker
    #arr_ord_cols = [6,27,28,29] #1-based index  twitter
    #arr_ord_cols = [1,2] #1-based index  mixed
    #arr_ord_cols = [8,28,29,30] #1-based index  trip adv
    #arr_ord_cols = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28] #1-based index pakdd
    #arr_ord_cols = [2,3,4,5,6,7,8,9,10,11,12,13,19] #1-based index hepatitis
    
    X_min_for_regress = None
    
    #filename = "noaa.csv"
    time_constant = 1.5 # Time constant controlling the rate of decay for linear models
    time_constant_ensemble_weight = 1.5 #ime constant controlling the rate of decay for ensemble weight
    
    f_arr = filename.split(".")

    filename = stream_folder  + filename

    Xy, X, y = read_data(filename)
    # Set x,y as numeric
    X = X.astype(float)
    n_samples, n_features = X.shape
    print(n_samples)
    print(n_features)
    ups =  np.count_nonzero(y == 1)
    downs = np.count_nonzero(y == 0)
    print("1s:",str(ups/n_samples))
    print("0s:",str(downs/n_samples))

    #create the minority buffer structure

    cols_str = []
    for f in range(1,n_features+1):
        cols_str.append(str(f))
    cols_str.append("life")
    cols_str.append("recall_weight")
    cols_str.append("total_weight")
    df_buffer_x = pd.DataFrame(columns = cols_str)
    #print("created")
    #print(df_buffer_x)
    
    
    cols_str = []
    cols_str.append('i') 
    for f in range(1,win_weight_count+1):
        cols_str.append(str(f))    
    df_weights_history = pd.DataFrame(columns = cols_str)

    
    all_models = {}
    all_models_df = pd.DataFrame( columns =['model_id','ind','dep','mse','rmse','r2','window','life','coef','intercept','weight'])

  
    arr_recall_0= []
    arr_recall_1= []
    arr_gmean= []
    arr_accuracy= []
    arr_kappa= []
    rl_pruned_ids = []

    col_means = []
    minority_buffer_x = []
    minority_buffer_y = []    
    
    total_miss_count = 0 
    imputed_from_arm = 0

 
    file_out_prefix = "BININI_" + str(f_arr[0])
    print(file_out_prefix)
    print("iteration:",k) 
    clfs = [DetectorClassifier(HoeffdingTreeClassifier(),[0,1])]
    clfs_label = ["Hoeffding Tree Classifier"]

    print("Features Originally:", n_features)

    for i in range(len(clfs)):
        #print("\n{}:".format(clfs_label[i]))
        with np.errstate(divide='ignore', invalid='ignore'):
            #print("Sending Features", n_features)
            print(disjunct_threshold_perc_maj)
            #print("....")
            file_out_prefix1 = file_out_prefix + "_" + str(k)
            r0,r1,gm = prequential(w,Xy,X, y, clfs[i], n_features,overlap_threshold_f1,
                        imbalance_ratio_threshold,disjunct_threshold_perc_maj,file_out_prefix1,n_train)
            arr_all_gmeans.append(gm)
            arr_all_recall0.append(r0)
            arr_all_recall1.append(r1)
    #print(file_out_prefix)
    #print("rules generated:",len(all_rules_df))
    #print("total missing:",total_miss_count)
    #print("imputed with arm:",imputed_from_arm)
    if(total_miss_count >0):
        print("% replaced:",round(100*(imputed_from_arm/total_miss_count),2))
        arr_all_perc_replaced.append(round(100*(imputed_from_arm/total_miss_count),2))
    else:
        arr_all_perc_replaced.append(0)
    total_miss_count = 0 
    imputed_from_arm = 0
    print("Current Avg Recall0 of all iterations:",np.mean(arr_all_recall0))
    print("Current Avg Recall1 of all iterations:",np.mean(arr_all_recall1))
    print("Current Avg GM of all iterations:",np.mean(arr_all_gmeans))
    #np.savetxt(output_folder + file_out_prefix + "-" + str(k) +'-ruleshistory.csv',  arr_rules_history, delimiter=',',fmt='%1.2f')
    

str_results = ""
str_results += "\n\n\n--------All Results------"
str_results += "\n" + file_out_prefix + "\n\n"
for a in range (0,len(arr_all_gmeans)):    
    str_results += "\nIteration:"+ str(a)
    str_results += "\nRecall0:"+ str(arr_all_recall0[a])
    str_results += "\nRecall1:"+ str(arr_all_recall1[a])
    str_results += "\nGM:"+ str(arr_all_gmeans[a])
    str_results += "\nReplaced %"+ str(arr_all_perc_replaced[a])

str_results += "\n------"

str_results += "\nIterations:" + str(iterations)
str_results += "\nAvg Recall0:"+ str(np.mean(arr_all_recall0))
str_results += "\nAvg Recall1:"+ str(np.mean(arr_all_recall1))
str_results += "\nAvg GM:"+ str(np.mean(arr_all_gmeans))
str_results += "\nAvg Replaced %"+ str(np.mean(arr_all_perc_replaced))
#with open(output_folder + file_out_prefix+ "-"  + '-results.txt','w') as file:
    #file.write(str_results)
    
print(str_results)
sys.exit()    


